# Experimento

## Importação de dados de artistas

### Dataset aset400
Eles utilizaram a versão ** filtrada ** do dataset.

In [1]:
# loading the artist ids
with open('aset400.3-canon-musicseer.ids', 'r') as f:
    artist_names = {}
    for i in f.read().split("\n"):
        name, id = i.partition(" ")[::2]
        artist_names[id] = name
    print("Total imported: %i artists" % len(artist_names.keys()))

Total imported: 401 artists


In [2]:
# loading filtered triplets
keys = artist_names.keys()
with open('filtered-survey.txt', 'r') as f:
    artist_triplets = []
    for i in f.read().split("\n"):
        trip = i.split(" ")[-3:]
        if (trip[0] != ''):
            if (trip[0] in keys) & (trip[1] in keys) & (trip[2] in keys):
                 artist_triplets.append(trip)
                
    print("Imported: %i triplets" % len(artist_triplets))
    


Imported: 16243 triplets
